In [1]:
import pandas as pd
import numpy as np

from surprise import Dataset
from surprise import SVD
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

### Modeling

In [2]:
#importing data set ("rating_C")
df = pd.read_csv('rating_C.csv',index_col=0)
df.head()

C:\Users\Vasin\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,anime_id,rating
0,1,20,0
1,1,24,0
2,1,79,0
3,1,226,0
4,1,241,0


* we split rating 0 out and act like its doesn't seen data

In [3]:
unseen_data = df[df['rating']==0]
# know_data using as data set
know_data = df[df['rating']!=0]
know_data.head(10)

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9


##### Modeling by original rating

In [4]:
# importing dataset(know_data) to surprise reader
reader = Reader(rating_scale=(1,10))
dataset = Dataset.load_from_df(know_data[['user_id','anime_id','rating']],reader=reader)

# split data to trainset and testset
trainset, testset = train_test_split(dataset,test_size=0.3,random_state=42)

# createing a model and fit the trainset with
model = SVD()
model.fit(trainset)

# predicting on testset and evaluate 
predict = model.test(testset)

In [5]:
# evaluate the RMSE
accuracy.rmse(predict)
accuracy.mae(predict)

RMSE: 1.1478
MAE:  0.8575


0.8575460650334777

RMSE seem pretty much than movie-len dataset, thus it depend on how spare data is.

### Recommen top-N recommendation for each user

In [6]:
from collections import defaultdict

In [7]:
ani = pd.read_csv('anime_C.csv',index_col=0)
ani = ani[['anime_id','name']]
ani.head()

,anime_id,name
0,32281,Kimi no Na wa.
1,5114,Fullmetal Alchemist: Brotherhood
2,28977,Gintama°
3,9253,Steins;Gate
4,9969,Gintama&#039;


In [8]:
ani_list = ani.set_index('anime_id').T.to_dict('list')
ani_list

{32281: ['Kimi no Na wa.'],
 5114: ['Fullmetal Alchemist: Brotherhood'],
 28977: ['Gintama°'],
 9253: ['Steins;Gate'],
 9969: ['Gintama&#039;'],
 32935: ['Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou'],
 11061: ['Hunter x Hunter (2011)'],
 820: ['Ginga Eiyuu Densetsu'],
 15335: ['Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare'],
 15417: ['Gintama&#039;: Enchousen'],
 4181: ['Clannad: After Story'],
 28851: ['Koe no Katachi'],
 918: ['Gintama'],
 2904: ['Code Geass: Hangyaku no Lelouch R2'],
 28891: ['Haikyuu!! Second Season'],
 199: ['Sen to Chihiro no Kamikakushi'],
 23273: ['Shigatsu wa Kimi no Uso'],
 24701: ['Mushishi Zoku Shou 2nd Season'],
 12355: ['Ookami Kodomo no Ame to Yuki'],
 1575: ['Code Geass: Hangyaku no Lelouch'],
 263: ['Hajime no Ippo'],
 44: ['Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen'],
 1: ['Cowboy Bebop'],
 30276: ['One Punch Man'],
 164: ['Mononoke Hime'],
 7311: ['Suzumiya Haruhi no Shoushitsu'],
 17074: ['Monogatari Series: Second 

In [13]:
show = []
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predict, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    #print(uid, [iid for (iid, _) in user_ratings])
    result = (uid, [ani_list[iid] for (iid, _) in user_ratings])
    show.append(result)

In [21]:
pd.set_option('display.max_colwidth',-1)
df3 = pd.DataFrame(show,columns=[['userID','The animes that recommended to user']])
df3.head(10)

<ipython-input-21-ea522fa8c485>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


,userID,The animes that recommended to user
0,51593,"[[Ginga Eiyuu Densetsu], [Baccano!], [Hunter x Hunter (2011)], [Mononoke], [Kino no Tabi: The Beautiful World]]"
1,46120,"[[Clannad], [Clannad: After Story], [Code Geass: Hangyaku no Lelouch], [Full Metal Panic!], [Katanagatari]]"
2,70507,"[[Steins;Gate], [Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare], [Gintama], [Gintama&#039;: Enchousen], [Shingeki no Kyojin]]"
3,56817,"[[Toaru Majutsu no Index II], [Gintama°], [Code Geass: Hangyaku no Lelouch], [Kimi no Na wa.], [Toaru Majutsu no Index: Endymion no Kiseki]]"
4,60015,"[[Zero no Tsukaima: Princesses no Rondo], [Angel Beats!], [Zero no Tsukaima: Futatsuki no Kishi], [Zero no Tsukaima], [Nogizaka Haruka no Himitsu]]"
5,38135,"[[Neon Genesis Evangelion], [Steins;Gate], [Cowboy Bebop], [Kara no Kyoukai 5: Mujun Rasen], [Kiseijuu: Sei no Kakuritsu]]"
6,62881,"[[Gintama], [Naruto], [Fairy Tail], [High School DxD], [Slam Dunk]]"
7,64898,"[[K], [Little Busters!: Refrain], [Kuroko no Basket], [Kanojo x Kanojo x Kanojo: Sanshimai to no DokiDoki Kyoudou Seikatsu], [Machine-Doll wa Kizutsukanai]]"
8,65081,"[[Code Geass: Hangyaku no Lelouch], [Steins;Gate], [Fullmetal Alchemist: Brotherhood], [Psycho-Pass], [Monogatari Series: Second Season]]"
9,60721,"[[Aria The Origination], [Steins;Gate], [Shirobako], [Zan Sayonara Zetsubou Sensei], [Zan Sayonara Zetsubou Sensei Bangaichi]]"
